In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
# train_df_path = "data/train.csv"
# test_df_path = "data/test.csv"
# train_df = pd.read_csv(train_df_path)
# test_df = pd.read_csv(test_df_path)
new_df_path = "/Users/user/Documents/Coding/cro_location_intelligence/src/data/all_data_embedding.csv"
df = pd.read_csv(new_df_path)
df.head()

,store_id,sum_pop_in_voronoi,voronoi_area,voronoi_density,cro_open_year,CRO_store_length,CRO_store_parking,CRO_store_stock_area,CRO_store_total_area,CRO_store_width,...,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511,embedding_512
0,1025.0,16369.32,38412.2,0.426149,16.0,20.00,0.0,97.39,274.39,14.00,...,0.618719,0.397206,0.539024,0.034935,-0.425025,-0.202426,-0.603762,-0.113011,0.282425,-0.264580
1,8396.0,15736.69,241705.5,0.065107,16.0,15.35,0.0,44.86,175.86,19.80,...,0.853443,0.604886,0.487524,0.250037,-0.343431,0.215231,-0.492256,-0.528084,-0.124986,-0.288903
2,8572.0,16156.14,193980.1,0.083288,16.0,9.85,0.0,12.90,145.90,17.80,...,0.461060,0.396897,0.465704,0.372753,0.014367,0.028150,-0.261129,-0.084676,-0.178961,0.154477
3,1058.0,11213.96,77343.2,0.144990,16.0,13.94,0.0,11.83,93.83,7.90,...,0.755208,0.682671,0.673349,-0.079623,-0.562957,-0.830060,-0.801435,-0.366279,0.880094,0.253283
4,1095.0,16216.47,90980.3,0.178242,16.0,15.82,0.0,35.18,113.18,7.94,...,0.432766,0.767998,0.388810,0.492982,0.151838,-0.086584,-0.651078,-0.546258,-0.215056,-0.015719


In [3]:
# use embedding to predict mockup_sale with xgboost
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np

# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

# get all column has embedding_ prefix
# feature_columns = [col for col in df.columns if col.startswith("embedding_")]
# feature_columns += ["subset"]
# feature_columns += ["y_nor"]
# assign to x
# all_data = df[feature_columns]

In [4]:
# normalize every column
from sklearn.preprocessing import StandardScaler

all_data = df.copy()
all_data = all_data.drop(columns=["store_id"])
all_data = all_data.drop(columns=["y_nor"])
train_all_data = all_data[all_data.subset == "train"]
all_data = all_data.drop(columns=["subset"])
train_all_data = train_all_data.drop(columns=["subset"])
scaler = StandardScaler()
scaler.fit(train_all_data)
all_data = scaler.transform(all_data)
all_data = pd.DataFrame(all_data)
# all_data.columns = feature_columns
# all_data["store_id"] = df["store_id"]
all_data["y_nor"] = df["y_nor"]
all_data["subset"] = df["subset"]
all_data.head()

,0,1,2,3,4,5,6,7,8,9,...,886,887,888,889,890,891,892,893,y_nor,subset
0,0.809053,-0.435626,1.276019,-0.585552,1.119992,-0.60246,1.958702,1.316572,-0.099400,1.694327,...,0.457054,-0.610891,-0.983325,-0.104843,-0.690702,0.676902,-0.050310,-0.970355,0.226238,train
1,0.587070,-0.246327,-0.271598,-0.585552,0.117934,-0.60246,-0.168104,-0.166140,0.836214,1.078724,...,0.214278,0.107851,-0.604111,1.191268,-0.142626,-0.891032,-1.833997,-1.052687,0.182798,train
2,0.734251,-0.290767,-0.193664,-0.585552,-1.067295,-0.60246,-1.462083,-0.616988,0.513588,-0.768087,...,0.111415,0.517898,1.058781,0.610704,0.993412,0.783935,-2.070301,0.448145,0.433002,train
3,-0.999912,-0.399375,0.070823,-0.585552,-0.185915,-0.60246,-1.505405,-1.400555,-1.083407,0.463120,...,1.090279,-0.993677,-1.624377,-2.052572,-1.662305,-0.279813,2.566342,0.782599,0.221966,train
4,0.755420,-0.386677,0.213359,-0.585552,0.219217,-0.60246,-0.560023,-1.109370,-1.076954,0.463120,...,-0.251072,0.919632,1.697689,0.254650,-0.923269,-0.959682,-2.228328,-0.127967,0.149573,train


In [5]:
# create noise data
def create_noise_data(data, times_sample, noise_level=0.1):
    new_data = data.copy()
    assert times_sample > 0 and isinstance(times_sample, int)
    for _ in range(times_sample):
        noise = np.random.normal(0, noise_level, data.shape)

        noise_data = data + noise
        new_data = pd.concat([new_data, noise_data])
    return new_data

In [6]:
X = all_data[all_data.subset == "train"]
X = X.drop(columns=["subset"])
y = X["y_nor"]
X_test = all_data[all_data.subset == "test"]
X_test = X_test.drop(columns=["subset"])
y_test = X_test["y_nor"]
X = X.drop(columns=["y_nor"])
X_test = X_test.drop(columns=["y_nor"])


# Assuming you have X, X_valid, y, y_valid
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.5, random_state=42
)
print(X_train.shape)
print(X_valid.shape)
train_noise = 0.25
valid_noise = 0.1
X_train = create_noise_data(X_train, 20, noise_level=train_noise)
# X_valid = create_noise_data(X_valid, 10, noise_level=train_noise / 10)
y_train = create_noise_data(y_train, 20, noise_level=valid_noise)
# y_valid = create_noise_data(y_valid, 10, noise_level=valid_noise / 10)
print(X_train.shape)
print(X_valid.shape)

(488, 894)
(489, 894)
(10248, 894)
(489, 894)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import ExtraTreesRegressor

# Assuming train_df and test_df are your training and testing DataFrames

# Separate features (X) and target variable (y) in the training set
# X_train = train_df.drop(["y_nor", "store_id"], axis=1)
# y_train = train_df["y_nor"]

# Separate features (X) and target variable (y) in the test set
# X_test = test_df.drop(["y_nor", "store_id"], axis=1)


def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 1000),
        # "n_estimators": trial.suggest_int("n_estimators", 500, 5000),
        # "max_depth": trial.suggest_int("max_depth", 3, 15),
        "max_depth": trial.suggest_int("max_depth", 20, 40),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
        "max_features": trial.suggest_float("max_features", 0.1, 1.0),
    }

    model = ExtraTreesRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    # rmse = mean_squared_error(y_valid, y_pred, squared=False)
    val_df = pd.DataFrame()
    val_df["y_nor"] = y_valid
    val_df["predicted_y_nor"] = y_pred
    val_df["mape"] = abs(val_df["y_nor"] - val_df["predicted_y_nor"]) / val_df["y_nor"]
    val_correct = val_df[val_df["mape"] < 0.15].shape[0]
    avg_mape = val_df["mape"].mean()
    loss = -avg_mape + (val_correct / val_df.shape[0])
    return loss


# Create a study object and optimize the objective function using Optuna
study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10, n_jobs=-1)
study.optimize(objective, n_trials=1)

[I 2023-12-21 09:24:22,800] A new study created in memory with name: no-name-ecb85439-2731-46cb-b461-777e4352d636
[I 2023-12-21 09:27:19,875] Trial 0 finished with value: -0.008386096916071228 and parameters: {'n_estimators': 791, 'max_depth': 37, 'min_samples_split': 8, 'min_samples_leaf': 12, 'max_features': 0.3053451340149207}. Best is trial 0 with value: -0.008386096916071228.


In [8]:
best_params = study.best_params
print("score", study.best_value)
print("Best Hyperparameters:", best_params)
display(best_params)

score -0.008386096916071228
Best Hyperparameters: {'n_estimators': 791, 'max_depth': 37, 'min_samples_split': 8, 'min_samples_leaf': 12, 'max_features': 0.3053451340149207}


{'n_estimators': 791,
 'max_depth': 37,
 'min_samples_split': 8,
 'min_samples_leaf': 12,
 'max_features': 0.3053451340149207}

In [9]:
import xgboost as xgb

# Get the best hyperparameters from the study

# Create the final model using the best hyperparameters
final_model = ExtraTreesRegressor(**best_params, random_state=42)
# final_model = ExtraTreesRegressor()
# final_model = xgb.XGBRegressor(**best_params, random_state=42)
# final_model = xgb.XGBRegressor()

# Train the final model on the entire training set
final_model.fit(X, y)
# final_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = final_model.predict(X_test)

In [10]:
len(y_pred_test)

68

In [11]:
y_pred_test = final_model.predict(X_test)
y_pred_train = final_model.predict(X)
y_pred_valid = final_model.predict(X_valid)
test_df = X_test.copy()
test_df["y_nor"] = y_test
train_df = X.copy()
train_df["y_nor"] = y
val_df = X_valid.copy()
val_df["y_nor"] = y_valid

# Add the predictions to the test_df DataFrame
test_df["predicted_y_nor"] = y_pred_test

train_df["predicted_y_nor"] = y_pred_train
val_df["predicted_y_nor"] = y_pred_valid
# Map store_id back to the test_df DataFrame

In [12]:
from sklearn.metrics import mean_absolute_percentage_error

# Assuming you have the actual target values for the test set in y_test
y_test = test_df["y_nor"]

# Extract predicted values from the DataFrame
predictions = test_df["predicted_y_nor"]
test_df["mape"] = abs(test_df["y_nor"] - test_df["predicted_y_nor"]) / test_df["y_nor"]


selected_columns = ["y_nor", "predicted_y_nor", "mape"]
test_df[selected_columns].head()
# count mape<0.15
test_correct = test_df[test_df["mape"] < 0.15].shape[0]
test_df.shape[0]
print("test_correct", test_correct)
print(f"accuracy {test_correct / test_df.shape[0]:.2f}")

test_correct 17
accuracy 0.25


In [13]:
# val
val_df["mape"] = abs(val_df["y_nor"] - val_df["predicted_y_nor"]) / val_df["y_nor"]
val_correct = val_df[val_df["mape"] < 0.15].shape[0]
val_df.shape[0]
print("val_correct", val_correct)
print(f"accuracy {val_correct / val_df.shape[0]:.2f}")

val_correct 225
accuracy 0.46


In [14]:
# train
train_df["mape"] = (
    abs(train_df["y_nor"] - train_df["predicted_y_nor"]) / train_df["y_nor"]
)
train_correct = train_df[train_df["mape"] < 0.15].shape[0]
train_df.shape[0]
print("train_correct", train_correct)
print(f"accuracy {train_correct / train_df.shape[0]:.2f}")

train_correct 448
accuracy 0.46


In [15]:
import math

for i in range(10):
    print(int(i + 0.5))

0
1
2
3
4
5
6
7
8
9


In [16]:
int(1.5), int(2.5), int(3.5), int(4.5), int(5.5), int(6.5), int(7.5), int(8.5), int(
    9.5
), int(10.5)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [17]:
math.floor(1.5), math.floor(2.5), math.floor(3.5), math.floor(4.5), math.floor(
    5.5
), math.floor(6.5), math.floor(7.5), math.floor(8.5), math.floor(9.5), math.floor(10.5)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [18]:
math.ceil(1.5), math.ceil(2.5), math.ceil(3.5), math.ceil(4.5), math.ceil(
    5.5
), math.ceil(6.5), math.ceil(7.5), math.ceil(8.5), math.ceil(9.5), math.ceil(10.5)

(2, 3, 4, 5, 6, 7, 8, 9, 10, 11)

In [19]:
round(1.5), round(2.5), round(3.5), round(4.5), round(5.5), round(6.5), round(
    7.5
), round(8.5), round(9.5), round(10.5)

(2, 2, 4, 4, 6, 6, 8, 8, 10, 10)